In [1]:
import rpy2
import rpy2.robjects as robjects
r = robjects.r
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import statsmodels.api as sm
import pandas as pd
from itertools import combinations

# Download Carlson Data

In [2]:
base = importr('base')
utils = importr('utils')
r('install.packages("FindIt", repos="https://CRAN.R-project.org/")')
findit = importr('FindIt')
'''using data from example in FindIt'''
robjects.r('data("Carlson", package = "FindIt")')
carlson = robjects.r['Carlson']
carlson_df = pandas2ri.rpy2py(carlson)
print(carlson_df)

'''trying a couple variables to use the 2 way amie'''
new_Record = robjects.r('''
Carlson$newRecordF<- factor(Carlson$newRecordF,ordered=TRUE,
levels=c("YesLC", "YesDis","YesMP",
"noLC","noDis","noMP","noBusi"))''')
promise = robjects.r('''
Carlson$promise <- factor(Carlson$promise,ordered=TRUE,levels=c("jobs","clinic","education"))
''')
model = robjects.r('''

fit2 <- CausalANOVA(formula=won ~ newRecordF + promise,
int2.formula = ~ newRecordF:promise,
data=Carlson, pair.id=Carlson$contestresp,diff=TRUE,
cluster=Carlson$respcodeS, nway=2)
''')

R[write to console]: trying URL 'https://CRAN.R-project.org/bin/macosx/big-sur-arm64/contrib/4.3/FindIt_1.2.0.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 430138 bytes (420 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console


The downloaded binary packages are in
	/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T//RtmpwUf5No/downloaded_packages
      won  newRecordF  promise  coeth_voting  relevantdegree  respcodeS   
1       1           6        1             2               1          1  \
2       0           7        3             1               2          1   
3       1           7        3             1               2          1   
4       0           3        2             1               2          1   
5       0           7        1             1               1          1   
...   ...         ...      ...           ...             ...        ...   
3228    1           4        1             1               2        544   
3229    1           5        1             2               2        544   
3230    0           2        2             1               1        544   
3231    0           7        1             2               2        544   
3232    1           3        1             1          

In [3]:
#One Hot Encode
enc = OneHotEncoder(handle_unknown='ignore')
X = carlson_df.iloc[:,1:-2]
y = carlson_df.iloc[:,0].to_numpy() *2 - 1
trans_x = enc.fit_transform(X).toarray() *2 - 1

#0-6 are record, 
print(trans_x)

[[-1. -1. -1. ...  1.  1. -1.]
 [-1. -1. -1. ... -1. -1.  1.]
 [-1. -1. -1. ... -1. -1.  1.]
 ...
 [-1.  1. -1. ... -1.  1. -1.]
 [-1. -1. -1. ...  1. -1.  1.]
 [-1. -1.  1. ... -1.  1. -1.]]
